In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [6]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

In [9]:
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 527 ( 1.5%) Users that have less than 1 test interactions


In [10]:
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from libs.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

In [11]:
def objective_function_P3_beta(optuna_trial):
       
        
    recommender_instance = RP3betaRecommender(URM_train_val)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 2, 40),
                             alpha = optuna_trial.suggest_float("alpha", 0.2, 0.5),
                             beta = optuna_trial.suggest_float("beta", 0.1, 0.3),
                             normalize_similarity = optuna_trial.suggest_categorical('normalize_similarity', [True, False]),
                             implicit = optuna_trial.suggest_categorical('implicit', [True, False])
                            )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance) 
    
    # Save MAP alongside RECALL
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["MAP"]

In [10]:
def objective_function_P3_alpha(optuna_trial):
    recommender_instance = P3alphaRecommender(URM_train)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 2, 50),
                             alpha = optuna_trial.suggest_float("alpha",0.2, 0.7, log=False),
                             normalize_similarity= True
                            )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)  
    
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["MAP"]

In [12]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [13]:
optuna_study_beta = op.create_study(direction="maximize")
        
save_results_beta = SaveResults("result_experiments/RP3beta/results_optuna_map.csv")
        
optuna_study_beta.optimize(objective_function_P3_beta,
                      callbacks=[save_results_beta],
                      n_trials = 500)

[I 2025-01-06 23:54:23,379] A new study created in memory with name: no-name-5b3aba08-4f54-4bb2-9dd9-6a67d9afc437


RP3betaRecommender: Similarity column 38121 (100.0%), 3799.47 column/sec. Elapsed time 10.03 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 9.71 sec. Users per second: 3628


[I 2025-01-06 23:54:43,650] Trial 0 finished with value: 0.031304758613755465 and parameters: {'topK': 29, 'alpha': 0.23305880505601254, 'beta': 0.18924077070615128, 'normalize_similarity': False, 'implicit': False}. Best is trial 0 with value: 0.031304758613755465.


RP3betaRecommender: Similarity column 38121 (100.0%), 3665.48 column/sec. Elapsed time 10.40 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 9.66 sec. Users per second: 3645


[I 2025-01-06 23:55:04,228] Trial 1 finished with value: 0.030822800988382824 and parameters: {'topK': 28, 'alpha': 0.23146757332049514, 'beta': 0.1580950906437779, 'normalize_similarity': False, 'implicit': False}. Best is trial 0 with value: 0.031304758613755465.


...

[I 2025-01-07 02:39:45,706] Trial 497 finished with value: 0.0428348677083358 and parameters: {'topK': 11, 'alpha': 0.334593058920543, 'beta': 0.2450383026985099, 'normalize_similarity': True, 'implicit': True}. Best is trial 322 with value: 0.04293583733776419.


RP3betaRecommender: Similarity column 38121 (100.0%), 3598.91 column/sec. Elapsed time 10.59 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 8.74 sec. Users per second: 4027


[I 2025-01-07 02:40:05,406] Trial 498 finished with value: 0.04271135807177545 and parameters: {'topK': 10, 'alpha': 0.27902356422737273, 'beta': 0.2408176834120574, 'normalize_similarity': True, 'implicit': True}. Best is trial 322 with value: 0.04293583733776419.


RP3betaRecommender: Similarity column 38121 (100.0%), 3521.74 column/sec. Elapsed time 10.82 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 8.86 sec. Users per second: 3972


[I 2025-01-07 02:40:25,495] Trial 499 finished with value: 0.042828863877244515 and parameters: {'topK': 12, 'alpha': 0.30713649581819324, 'beta': 0.2530503326029971, 'normalize_similarity': True, 'implicit': True}. Best is trial 322 with value: 0.04293583733776419.


In [15]:
optuna_study_beta.best_params, optuna_study_beta.best_value

({'topK': 11,
  'alpha': 0.22265853091374718,
  'beta': 0.25109461460614335,
  'normalize_similarity': True,
  'implicit': True},
 0.04293583733776419)

In [12]:
# save_results_beta.results_df.to_csv("result_experiments/RP3beta/results_optuna_20.csv")

In [12]:
optuna_study_alpha = op.create_study(direction="maximize")
        
save_results_alpha = SaveResults("result_experiments/RP3alpha/results_optuna_map.csv")
        
optuna_study_alpha.optimize(objective_function_P3_alpha,
                      callbacks=[save_results_alpha],
                      n_trials = 300)

[I 2025-01-05 22:29:33,729] A new study created in memory with name: no-name-45826424-b985-4564-a578-c456c1e53536


P3alphaRecommender: Similarity column 38121 (100.0%), 5340.98 column/sec. Elapsed time 7.14 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.19 sec. Users per second: 4238


[I 2025-01-05 22:29:49,349] Trial 0 finished with value: 0.026831234711114888 and parameters: {'topK': 8, 'alpha': 0.6094379801732381}. Best is trial 0 with value: 0.026831234711114888.


P3alphaRecommender: Similarity column 38121 (100.0%), 5037.19 column/sec. Elapsed time 7.57 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.85 sec. Users per second: 3926


[I 2025-01-05 22:30:06,391] Trial 1 finished with value: 0.02680136996632266 and parameters: {'topK': 44, 'alpha': 0.5657407188050275}. Best is trial 0 with value: 0.026831234711114888.


...

[I 2025-01-05 23:52:55,712] Trial 297 finished with value: 0.02689011405644884 and parameters: {'topK': 38, 'alpha': 0.31462227817665916}. Best is trial 124 with value: 0.027814196860498817.


P3alphaRecommender: Similarity column 38121 (100.0%), 4858.46 column/sec. Elapsed time 7.85 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 9.05 sec. Users per second: 3837


[I 2025-01-05 23:53:12,965] Trial 298 finished with value: 0.027750966239217065 and parameters: {'topK': 15, 'alpha': 0.2978324041299271}. Best is trial 124 with value: 0.027814196860498817.


P3alphaRecommender: Similarity column 38121 (100.0%), 4822.71 column/sec. Elapsed time 7.90 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.98 sec. Users per second: 3866


[I 2025-01-05 23:53:30,162] Trial 299 finished with value: 0.027480684356979413 and parameters: {'topK': 10, 'alpha': 0.3108702935429274}. Best is trial 124 with value: 0.027814196860498817.


In [13]:
optuna_study_alpha.best_params

{'topK': 14, 'alpha': 0.30489731750244276}

In [17]:
save_results_alpha.results_df.to_csv("result_experiments/RP3alpha/results_optuna.csv")

In [17]:
results_alpha = pd.read_csv("result_experiments/RP3alpha/results_optuna.csv")
results_beta = pd.read_csv("result_experiments/RP3beta/results_optuna_50.csv")

In [18]:
results_beta.sort_values(by=['result'], ascending=False)

,Unnamed: 0,topK,alpha,beta,result
235,235,12.0,0.349899,0.181734,0.055589
267,267,12.0,0.358275,0.183071,0.055585
232,232,12.0,0.352013,0.170365,0.055554
249,249,12.0,0.318763,0.178043,0.055539
265,265,12.0,0.359616,0.178041,0.055538
...,...,...,...,...,...
195,195,2.0,0.863639,0.172310,0.035370
7,7,29.0,0.579395,0.922559,0.034820
133,133,14.0,0.282290,0.985331,0.034568
9,9,48.0,0.293029,0.919261,0.033317


In [8]:
results_alpha.sort_values(by=['result'], ascending=False)

,Unnamed: 0,topK,alpha,result
101,101,13.0,0.358969,0.054205
264,264,13.0,0.356510,0.054203
280,280,13.0,0.359350,0.054201
97,97,13.0,0.356190,0.054192
267,267,13.0,0.355121,0.054187
...,...,...,...,...
154,154,2.0,0.350523,0.036901
213,213,2.0,0.350160,0.036896
220,220,2.0,0.350028,0.036895
248,248,2.0,0.350073,0.036895
